## Wysokopoziomowy interfejs Keras

### Importy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
print(f"TensorFlow version: {tf.__version__}")

### Hiperparametry modelu

In [ ]:
# MNIST dataset params
num_classes = 10
num_features = 784  # obrazki mają rozmiar 28 x 28 = 784

# Training parameters
learning_rate = 0.01
training_steps = 1000
batch_size = 256
display_step = 50

## E**T**L (Extract Transform Load)

## E z ETL

### [Zestaw danych MNIST](http://yann.lecun.com/exdb/mnist/)

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print(f"Dane treningowe: {len(x_train)} próbek")
print(f"Dane testowe: {len(x_test)} próbek")

In [ ]:
print(x_train[1])  # obrazki 24 x 24

In [ ]:
fig = plt.figure
plt.imshow(x_train[1], cmap='gray')
plt.show()

## T z ETL

In [ ]:
# Konwersja do float32
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)

# Spłaszczenie sygnału (28*28 = 784 cech)
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])

# Normalizacja sygnału (z zakresu [0, 255] do zakresu [0.0, 1.0])
x_train, x_test = x_train / 255., x_test / 255.

In [ ]:
# Do daleszej obróbki danych używamy modułu tf.data

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))  # zrobienie obiektów tf.Tensor z numpy array
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)  # przetasowanie z powtórzeniami (regularyzacja) 
# i zrobienie batchy

In [ ]:
# Przygotowujemy miejsce na parametry modelu

# Kształt wag to [784, 10] - 28*28 cech i 10 klas
W = tf.Variable(tf.ones([num_features, num_classes]), name='weight')

# Kształt bias to [10] - 10 klas
b = tf.Variable(tf.zeros([num_classes]), name='bias')

## Definicja modelu

In [ ]:
# model - regresja logistyczna (model liniowy przepuszczony przez znormalizowaną funkcję wykładniczą - softmax)
def logistic_regression(x):
  return tf.nn.softmax(tf.matmul(x, W) + b)

In [ ]:
# Funkcja kosztu - entropia krzyżowa
def categorical_cross_entropy(y_pred, y_true):
  
  # One-hot encoding
  y_true = tf.one_hot(y_true, depth=num_classes)
  
  # Clip - ograniczenie wartości, żeby uniknąć błędu log(0)
  y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
  
  # wzór na entropie krzyżową
  return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred), 1))

In [ ]:
# Metryka - dokładność
def accuracy(y_pred, y_true):
  
  # Zliczamy poprawne predykcje
  correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))  # argmax - indeks największego 
  # elementu wzdłuż wybranej osi (czyli tutaj, tego którego prawdopodbieństwo jest największe)
  
  return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
# Optymalizator - Stochastic Gradient Descent
optimizer = tf.optimizers.SGD(learning_rate)

def run_optimization(x, y):
  with tf.GradientTape() as g:
    pred = logistic_regression(x)
    loss = categorical_cross_entropy(pred, y)
  
  gradients = g.gradient(loss, [W, b])
  optimizer.apply_gradients(zip(gradients, [W, b]))

## Trenowanie modelu

In [ ]:
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
  run_optimization(batch_x, batch_y)
    
  if step % display_step == 0:
    pred = logistic_regression(batch_x)
    loss = categorical_cross_entropy(pred, batch_y)
    acc = accuracy(pred, batch_y)
    print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

In [ ]:
pred = logistic_regression(x_test)
print("Test Accuracy: %f" % accuracy(pred, y_test))

In [ ]:
# Popatrzmy na 5 pierwszych wyników
n_images = 5
test_images = x_test[:n_images]
predictions = logistic_regression(test_images)

for i in range(n_images):
  plt.imshow(np.reshape(test_images[i], [28, 28]), cmap='gray')
  plt.show()
  print("Model prediction: %i" % np.argmax(predictions.numpy()[i]))